# Sales dashboard 

### Interactive Sales Dashboard using Bokeh and Pandas

The **Bokeh** is an Open-Source library for interactive visualization that renders graphics using HTML and JavaScript. It is common used for visualization and can be support EDA analysis. It can also be used to build web-based dashboards and applications. The Bokeh library supports many charts such as line graphs, bar charts, pie charts, stacked bar charts, histograms, and scatter plots. 

For this analysis we used "Supermarket sales" dataset. Based on data analysis of this dataset we build interactive dashboard by using Bokeh library.

Firstly we start by installing the library **pandas-bokeh** using pip.

In [1]:
!pip install pandas-bokeh

### Importing libraries and data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import pandas_bokeh
from bokeh.io import output_file, show

In [2]:
pandas_bokeh.output_notebook() 

Loading BokehJS ...

In [3]:
df = pd.read_csv(r'C:\Python Scripts\Datasets\supermarket_sales.csv')
df.head()

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761905,16.2155,7.4
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,20:33,Ewallet,465.76,4.761905,23.2880,8.4
4,373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37,Ewallet,604.17,4.761905,30.2085,5.3


### Data cleaning and preparation

Changing names of selected column:

In [4]:
df.rename(columns={'gross margin percentage': 'gross margin %'}, inplace=True)

Replacing names of product line variable:

In [5]:
df.replace({'Product line': {'Electronic accessories': 'Electronics',
                                            'Fashion accessories': 'Fashion',
                                            'Food and beverages': 'Food',
                                            'Health and beauty': 'Health',
                                            'Home and lifestyle': 'Home',
                                            'Sports and travel': 'Sports'}}, inplace=True)

Changing Date variable:

In [6]:
df["Date"] = pd.to_datetime(df["Date"], errors='coerce')

In [7]:
df.Date.value_counts()

2019-02-07    20
2019-02-15    19
2019-01-08    18
2019-03-02    18
2019-03-14    18
              ..
2019-03-17     6
2019-02-01     6
2019-01-04     6
2019-03-21     6
2019-02-21     6
Name: Date, Length: 89, dtype: int64

In [8]:
df.head()

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin %,gross income,Rating
0,750-67-8428,A,Yangon,Member,Female,Health,74.69,7,26.1415,548.9715,2019-01-05,13:08,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronics,15.28,5,3.8200,80.2200,2019-03-08,10:29,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,A,Yangon,Normal,Male,Home,46.33,7,16.2155,340.5255,2019-03-03,13:23,Credit card,324.31,4.761905,16.2155,7.4
3,123-19-1176,A,Yangon,Member,Male,Health,58.22,8,23.2880,489.0480,2019-01-27,20:33,Ewallet,465.76,4.761905,23.2880,8.4
4,373-73-7910,A,Yangon,Normal,Male,Sports,86.31,7,30.2085,634.3785,2019-02-08,10:37,Ewallet,604.17,4.761905,30.2085,5.3


### Data analysis

**Build sales functions**

7-day moving average of daily sales:

In [9]:
days = (df["Total"].groupby(df['Date']).sum().rolling(7, min_periods=7).mean())

Income by city:

In [10]:
income_city = df.pivot_table(index='City',
                                values='gross income',
                                aggfunc='sum').round(0)

In [11]:
income_city

,gross income
City,
Mandalay,5057.0
Naypyitaw,5265.0
Yangon,5057.0


Sales by gender and product line:

In [12]:
total_gender = df.pivot_table(index='Gender',
                                columns='Product line',
                                values='Total',
                                aggfunc='sum').round(0)

Sales by City and Customer type:

In [13]:
total_customer = df.pivot_table(index='City',
                                columns='Customer type',
                                values='gross income',
                                aggfunc='sum').round(0)

In [14]:
total_customer

Customer type,Member,Normal
City,,
Mandalay,2557.0,2500.0
Naypyitaw,2709.0,2557.0
Yangon,2554.0,2503.0


The products generate the most income:

In [15]:
product = df.groupby('Product line')['gross income'].sum().sort_values(ascending=True).to_frame()
product

,gross income
Product line,
Health,2342.5590
Home,2564.8530
Fashion,2585.9950
Electronics,2587.5015
Sports,2624.8965
Food,2673.5640


### Plotting charts 

At this moment we can plot the charts in a dashboard. From above functions we create a folowing charts: line plot, two bar charts, two stacked bar chart and pie chart.

In [16]:
#color palette
colors=['#FDE724', '#D01C8B', '#4DAC26', '#d7191c']

In [17]:
# plot 1 - line plot
p_line = days.plot_bokeh(kind="line",y="Total",color='#d01c8b', title='7-day moving average of daily sales.', plot_data_points=True,show_figure=True)

In [18]:
# plot 2 - bar plot
p_bar = income_city.plot_bokeh(kind="bar", color='#d01c8b', legend=False, title='Revenue by the city.', show_figure=True)

In [19]:
# plot 3 - bar plot
p_bar2 = total_gender.plot_bokeh(kind="bar", legend=False, title='Sales by Product line and Gender.', show_figure=True)

In [20]:
# plot 4- stacked bar chart
p_stack = total_customer.plot_bokeh(kind='barh', stacked=True, title='Revenue by City and Customer type.', colormap=colors, show_figure=True)

In [21]:
# plot 5- pie chart
p_pie = df.groupby(["Product line"])["Total"].sum().plot_bokeh(kind='pie', y='Total', title='Total sales by products line.',show_figure=True) 

__x__values_original


In [22]:
# plot 6 - stacked bar chart
p_stack2 = product.plot_bokeh(kind='barh', stacked=True, color='#4DAC26',legend=False, title='The products which generate the most revenue.', show_figure=True)

Now we make Dashboard with Grid Layout: 

In [24]:
# Save the dashboard
output_file('sales.html')

In [25]:
plot = pandas_bokeh.plot_grid([[p_bar2, p_pie, p_line],[p_stack2, p_bar, p_stack]], plot_width=400)